# Imstallation et importation des packages 

In [ ]:
""" %pip install numpy
%pip install pandas
%pip install matplotlib
%pip install openpyxl
%pip install pycodestyle
%pip install pycountry
%pip install seaborn
%pip install pickleshare"""
import pycodestyle as pep8
import openpyxl as xl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pycountry
import seaborn as sns
np.random.seed(123)

# Importation de la BD

In [ ]:
rate=pd.read_excel("C:/Users/yousr/Downloads/Projet_py/Unemployment Rate.xlsx", sheet_name='monthly' , index_col=0 )
rate.head(5)
rate.index = pd.to_datetime(rate.index,format='%YM%m')
rate.index =rate.index.strftime('%Y-%m')

In [ ]:
# Noms des colonnes
col_names=rate.columns
print(col_names)

# Type de données

In [ ]:
# Type de données de chaque colonne 
datas_type= rate.dtypes
print(datas_type)

# Doublons

In [ ]:
duplicates = rate[rate.duplicated()]
duplicates.head(5) # 0 doublons dans notre DF

# Erreurs de frappes pour la colonne index

In [ ]:
rate.index = pd.to_datetime(rate.index, errors='coerce')
errors = rate[rate.index.isnull()]
errors.head(10) # Pas d'erreurs #########################################################################################

# Construction de DF

In [ ]:
# Garder uniquement les pays dans la DF : Méthode search_fuzzy de pycountry ###############################################################

def detect_countries(col_names):
    countries_detected = []
    
    for name in col_names:
        try:
            # Essayer de trouver le pays par son nom
            pays= pycountry.countries.search_fuzzy(name)[0]
            countries_detected.append(pays.name)
        except LookupError:
            # Ignorer les noms qui ne correspondent à aucun pays
            pass
    
    return countries_detected

# Liste des pays detectés

countries_detected = detect_countries(col_names)
print("Noms de pays détectés :", countries_detected)

# Longeur des listes : countries_detected et col_names
print(len(countries_detected))
print(len(col_names))

# Différence entre les deux listes pour obtenir les noms des colonnes ignorées par la fonction detect_countries
#L'objectif est de vérifier si des pays ont été omis par la fonction detect_countries

# Pays dans col_names et pas dans countries_detected
diff= list(set(col_names) - set(countries_detected))
print(diff) 

# Pays dans countries_detected et pas dans col_names
diff2= list(set(countries_detected)- set(col_names))
print(diff2)

countries_omitted =['Korea, Rep.', 'Taiwan, China', 'Hong Kong SAR, China', 'Czech Republic', 'Egypt, Arab Rep.', 'Venezuela, RB'] ####################""
concat = countries_detected + countries_omitted 

# Enfin, la liste finale des pays à retenir dans la DF
countries_detected =list(set(concat)- set(diff2))
print(countries_detected)

# La DF taux de chomage à retenir
rate_filter= rate[countries_detected]
rate_filter.columns

In [ ]:
# Pour faciliter la lecture de la DF, on remplace les noms des pays par leurs codes ISO  correspondant

# Dictionnaire de correspondance entre noms complets des pays et leurs codes
corresp = {country.name: country.alpha_3 for country in pycountry.countries}

# Liste initiale des noms complets des colonnes
country = countries_detected
# Transformation des noms complets des colonnes en abréviations
country_codes = [corresp.get(pays, pays) for pays in country]

# Remplacement des noms des pays par leurs codes dans la merged
rate_filter.columns=country_codes
rate_filter.columns
rate_filter.head(5)

#Ordre alphabétique des colonnes 
rate_filter_sort= rate_filter.sort_index(axis=1)
rate_filter_sort.head(5)

# Visualisation

In [ ]:
# Visualisation des valeurs manquantes NA
import missingno as msno
# Diagramme à barres des valeurs manquantes
msno.bar(rate_filter_sort)
plt.show()

# % des NA

In [ ]:
# Calculer le taux de valeurs manquantes dans chaque colonne
size = rate_filter_sort.shape
nan_sum = rate_filter_sort.isna().sum() # Total du nombre de NA par colonne
nan_percent = nan_sum.sort_values(ascending=True) * 100 / size[0]
print(nan_percent)

# Seuil des NA

In [ ]:
# Récupérer les colonnes avec un pourcentage de valeurs manquantes supérieur à 40%
cols_to_drop = nan_percent[nan_percent > 40].index
print(cols_to_drop)

# Supprimer les colonnes de la DataFrame
rate_filter_sort.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
# Type de données de chaque colonne 
data_type = rate_filter_sort.dtypes
print(data_type)

# Imputation

In [ ]:
# Imputation des NA

#rate_filter_sort_inter=rate_filter_sort.interpolate(method='linear', axis=0, limit=None, inplace=False, limit_direction=None, limit_area=None)

#rate_filter_sort.fillna(value=None, method='ffill', axis=None, inplace=False, limit=None)

rate_no_na = rate_filter_sort.fillna(rate_filter_sort.mean())

#rate_filter_sort_fill= rate_filter_sort.iloc[-5:].ffill()

indice_premiere_na = rate_filter_sort.isna().any(axis=1).idxmax()
print(indice_premiere_na)

#numero_ligne = rate_filter_sort.index.get_loc(indice_premiere_na) + 1
"""
from datetime import datetime
date_today = datetime.now()

# Extrait l'année de la date actuelle
current_year = date_today.year
str(current_year)
rate_filter_sort.index = pd.to_datetime(rate_filter_sort.index, format='mixed')


rate_filter_sort.index = rate_filter_sort.index.astype(str)
rate_current_year = rate_filter_sort[rate_filter_sort.index.str.contains(current_year)]


#rate_filter_sort.index = pd.to_datetime(rate_filter_sort.index)

#year2023 =rate_filter_sort.loc[rate_filter_sort.index.year == 2023]

#.fillna(rate_filter_sort.mean())
print(rate_filter_sort.index.dtype) """

# Valeurs abérrantes

In [ ]:
# Les valeurs abérrantes 
plt.figure(figsize=(10, 6))
sns.boxplot(data=rate_no_na)
plt.title('Boîtes à moustaches des séries temporelles')
plt.show()

# Ajustement

In [ ]:
rate_no_na.index = pd.to_datetime(rate_no_na.index, format='%Y-%m-%d')

# Grouper par année et vérifier si tous les mois sont présents
rate_12= rate_no_na.groupby(rate_no_na.index.year).filter(lambda x: len(x) == 12)
rate_12= pd.DataFrame(rate_12)
rate_12.head(5)

# Regroupement

In [ ]:
rate_quart = rate_12.resample('Q-JAN').mean()
#print(rate_quart)

# Jointure

In [ ]:
# Ignorer les jours dans l'index
rate_quart.index =rate_quart.index.strftime('%Y-%m')

# Importation de la DF gdp_no_na grace à pickleshare 
from pickleshare import PickleShareDB
db = PickleShareDB('~/data_store')
gdp_no_na= db['my_dataframe']

In [ ]:
# Fusionner les DataFrames sur l'index en utilisant une jointure interne (how='inner')
common_columns = rate_quart.columns.intersection(gdp_no_na.columns) # Pour garder uniquement les colonnes présentes dans les deux DF
merged= pd.merge(rate_quart[common_columns], gdp_no_na[common_columns], left_index=True, right_index=True, how='inner' )
merged.head(5)

In [ ]:
print(rate_quart.index)
print(gdp_no_na.index)
print(rate_quart.index.equals(gdp_no_na.index))

# DF classification

In [ ]:
mean_rate = pd.DataFrame(rate_quart.mean(), columns=['Mean'])
mean_gdp = pd.DataFrame(gdp_no_na.mean(), columns=['Mean'])
merged_mean= pd.merge(mean_rate, mean_gdp, left_index=True, right_index=True, how='inner')

# Pivotage

In [ ]:
#result_table = merged.melt(id_vars='merged.index', var_name='Country', value_name='Value')
# result_table = merged.pivot(index='gdp_no_na.index', columns='rate_quart', values='gdp_no_na')


In [ ]:
pivot_table_result = pd.pivot_table(merged, values=['Taux de Chômage', 'PIB'], index=['Pays'], columns=['Année'], aggfunc='mean')



# Utilisez la méthode melt pour fondre la DataFrame
melted_merged = merged.reset_index().melt(id_vars=['index'], var_name='Variable', value_name='Valeur')

# Renommez la colonne 'index' en 'Année'
melted_merged = melted_merged.rename(columns={'index': 'Année'})

# Séparez la colonne 'Variable' en deux colonnes 'Pays' et 'Indicateur'
melted_merged[['Pays', 'Indicateur']] = melted_merged['Variable'].str.split('_', expand=True)

# Supprimez la colonne 'Variable' maintenant inutile
melted_merged = melted_merged.drop(columns=['Variable'])

# Réglez l'index sur 'Année' si vous le souhaitez
melted_merged = melted_merged.set_index('Année')

# Affichez le résultat
print(melted_merged)